In [ ]:
# Import libraries
import json
from csv import writer
from apiclient.discovery import build
import pandas as pd
import pickle
import urllib.request
import urllib

In [ ]:
# API key from YouTube Data API key
key = <'Key'>
videoId = 'PKtnafFtfEo' # Youtube Rewind 2020, Thank God It's Over
channelId = 'UCX6OQ3DkcsbYNE6H8uQQuVA' # MrBeast 

In [ ]:
def build_service():
    YOUTUBE_API_SERVICE_NAME = "youtube"
    YOUTUBE_API_VERSION = "v3"
    return build(YOUTUBE_API_SERVICE_NAME,
                 YOUTUBE_API_VERSION,
                 developerKey=key)

In [ ]:
def get_comments(part='snippet', 
                 maxResults=100, 
                 textFormat='plainText',
                 order='time',
                 allThreadsRelatedToChannelId=channelId,
                 csv_filename="MrBeast_comments"
                 ):
  
  # Initialize empty lists which will store the stats on a video
  comments, commentsId, authorurls, authornames, repliesCount, likesCount, viewerRating, dates, vidIds, totalReplyCounts,vidTitles = [], [], [], [], [], [], [], [], [], [], []

  # Build the serivce
  service = build_service()

  # Call the API with the service
  response = service.commentThreads().list(
        part=part,
        maxResults=maxResults,
        textFormat='plainText',
        order=order,
        allThreadsRelatedToChannelId=channelId
  ).execute()

  # There is a limit to the amount of data I can pull. 
  # For MrBeast, his channel is quite large and active so this loop will stop running at the quota
  # Typically I would set a hard cap on the number of comments to pull that is lower than the quota to test if the function is working properly but I have a guide on this API sourced below.

  while response:
    for item in response['items']:
            # Index item for desired data features
            comment = item['snippet']['topLevelComment']['snippet']['textDisplay']
            comment_id = item['snippet']['topLevelComment']['id']
            reply_count = item['snippet']['totalReplyCount']
            like_count = item['snippet']['topLevelComment']['snippet']['likeCount']
            authorurl = item['snippet']['topLevelComment']['snippet']['authorChannelUrl']
            authorname = item['snippet']['topLevelComment']['snippet']['authorDisplayName']
            date = item['snippet']['topLevelComment']['snippet']['publishedAt']
            vidId = item['snippet']['topLevelComment']['snippet']['videoId']
            totalReplyCount = item['snippet']['totalReplyCount']
            vidTitle = get_vid_title(vidId)

            # Append to lists
            comments.append(comment)
            commentsId.append(comment_id)
            repliesCount.append(reply_count)
            likesCount.append(like_count)
            authorurls.append(authorurl)
            authornames.append(authorname)
            dates.append(date)
            vidIds.append(vidId)
            totalReplyCounts.append(totalReplyCount)
            vidTitles.append(vidTitle)

    try:
            if 'nextPageToken' in response:
                response = service.commentThreads().list(
                    part=part,
                    maxResults=maxResults,
                    textFormat=textFormat,
                    order=order,
                    allThreadsRelatedToChannelId=channelId,
                    pageToken=response['nextPageToken']
                ).execute()
            else:
                break
    except: break
  
  return {
        'comment': comments,
        'comment_id': commentsId,
        'author_url': authorurls,
        'author_name': authornames,
        'reply_count' : repliesCount,
        'like_count' : likesCount,
        'date': dates,
        'vidid': vidIds,
        'total_reply_counts': totalReplyCounts,
        'vid_title': vidTitles
    }

In [ ]:
def get_vid_title(vidid):
    # VideoID = "LAUa5RDUvO4"
    # Usually, the videoID is after the ?v=. This information can also be accessed when clicking share on the video itself.
    params = {"format": "json", "url": "https://www.youtube.com/watch?v=%s" % vidid}
    url = "https://www.youtube.com/oembed"
    query_string = urllib.parse.urlencode(params)
    url = url + "?" + query_string

    with urllib.request.urlopen(url) as response:
        response_text = response.read()
        data = json.loads(response_text.decode())
        return data['title']

In [ ]:
if __name__ == '__main__':
    MrBeast_comments = get_comments()
    df = pd.DataFrame(MrBeast_comments)
    print(df.shape)
    print(df.head())
    df['date'] = pd.to_datetime(df['date'], errors='coerce')
    df['just_date'] = df['date'].dt.date
    df.to_csv('./MrBeast_comments.csv')

(11258, 10)
                                             comment  ...                                          vid_title
0  "I'm sorry, international fans"\n\n*Cries in P...  ...   I Opened A Restaurant That Pays You To Eat At It
1                                          Relatable  ...  I Gave People $1,000,000 But ONLY 1 Minute To ...
2                      How do you Gus love that crap  ...                 I Went Back To 1st Grade For A Day
3  I wouldn’t be surprised if MrBeast became one ...  ...                    I Bought Everything In 5 Stores
4                               Chandler:eats a rose  ...  Surprising My Girlfriend With 100,000 Roses Fo...

[5 rows x 10 columns]
